

### **Lab 1.1? - Ngram model** - To comment - **Jonathan** Incorporate 2 new activities - random choice 2 + count zero **Annie**

[Possibly part of the Instructions Page]

**Introduction**

In this lab, we will build an n-gram model. An n-gram is a contiguous sequence of *n* words. An n-gram *model* uses these sequences to predict the probability of the next word given a preceding sequence of n-1 words (the context). We'll use the conditional probability formula:

 **Formula**

$$ P(\text{B} \mid \text{A}) = \frac{\text{Count(A B)}}{\text{Count(A)}} $$

where:

*  $P(B|A)$ is the probability of word B following word A.
* $\text{Count(A B)}$ is the number of times the sequence "A and B" appear together in the text.
* $\text{Count(A)}$ is the number of times word A appears in the text.

**Steps**

1. **Data Preparation:** Define your dataset and break the sentences into individual words (also called tokenization).

2. **N-gram Creation:**  Create n-grams from the tokenized words and calculate the numerator of our formula: $\text{Count(A B)}$.

3. **Probability Calculation:** Calculate the denominator: $\text{Count(A)}$. Divide the numerator (from Step 2) by the denominator to get $P(B|A)$.

4. **Text Generation:** Use the calculated probabilities to generate new text based on the n-gram model. The last step also involves a coding activity.

So let's begin!



**Step 1: Define dataset and break sentences into individual words.**

Let's define the corpus (text) we will use for this exercise. We will use the TinyStories dataset which comprises of synthetically generated short stories with a limited vocabulary.

*Run the following code cells by clicking on the play button*

In [1]:
from IPython.display import clear_output
# !pip install datasets
# from datasets import load_dataset
from collections import Counter, defaultdict
import pandas as pd
import random
from google.colab import drive
import sys
import os
import pandas as pd

clear_output()  # Clears the output

In [3]:
# @title Run me to mount drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

path='/content/drive/MyDrive/A&T_workshop/data'

import sys

sys.path.append(path)
import os
os.chdir(path)
!pwd

import pandas as pd

stories=pd.read_json('africa_galore.json')
stories =stories['description']
stories[0]

Mounted at /content/drive/


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/A&T_workshop/data'

In [ ]:
for story in stories:
  print(story)
  print('-------------------')

Didier Drogba, a name that resonates with football fans worldwide, is an Ivorian legend who transcended the sport to become a symbol of hope and unity for his nation. His powerful presence on the pitch, his clinical finishing, and his ability to rise to the occasion made him one of the most feared strikers of his generation.  Drogba's impact extended beyond club football.  He captained the Ivory Coast national team, leading them to multiple World Cup appearances and becoming their all-time leading scorer.  His influence helped bring a period of peace to his war-torn country, demonstrating the unifying power of sport.  More than just a footballer, Drogba is a humanitarian and a national icon, revered for his contributions both on and off the field.
-------------------
Samuel Eto'o, a Cameroonian footballing icon, is widely regarded as one of Africa's greatest players. His blistering pace, incredible skill, and prolific goalscoring record earned him accolades at the highest levels of the


**Step 2: Create ngrams out of words.**



In order to create ngrams, we must first convert sentences into individual words. This is also called tokenization.

In [ ]:
# Function to split sentences into individual words.
def split_text(text):
    return text.split()

split_text("Here's how you tokenize! Simple, isn't it?")

["Here's", 'how', 'you', 'tokenize!', 'Simple,', "isn't", 'it?']

In [ ]:
#Let's look at some ngrams in our data
all_unigrams=[]
all_bigrams=[]
all_trigrams=[]

def generate_ngrams(text, n):
    words = split_text(text)
    ngrams = [" ".join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams


for story in stories:
    all_unigrams.extend(generate_ngrams(story,1))
    all_bigrams.extend(generate_ngrams(story, 2))
    all_trigrams.extend(generate_ngrams(story, 3))


print("Unigrams:",all_unigrams[:10])
print("Bigrams:",all_bigrams[:10])
print("Trigrams:",all_trigrams[:10])

Unigrams: ['Didier', 'Drogba,', 'a', 'name', 'that', 'resonates', 'with', 'football', 'fans', 'worldwide,']
Bigrams: ['Didier Drogba,', 'Drogba, a', 'a name', 'name that', 'that resonates', 'resonates with', 'with football', 'football fans', 'fans worldwide,', 'worldwide, is']
Trigrams: ['Didier Drogba, a', 'Drogba, a name', 'a name that', 'name that resonates', 'that resonates with', 'resonates with football', 'with football fans', 'football fans worldwide,', 'fans worldwide, is', 'worldwide, is an']


Let's see which n-grams appear most frequently.

In [ ]:
bigram_counts = Counter(all_bigrams)
print("Most common Bigrams")
print(bigram_counts.most_common(10))

print("Most common Trigrams")
trigram_counts=Counter(all_trigrams)
print(trigram_counts.most_common(10))

Most common Bigrams
[('is a', 134), ('of the', 90), ('and the', 72), ('in the', 67), ('with a', 60), ('and a', 57), ('in a', 56), ('was a', 45), ('to the', 40), ('went looking', 39)]
Most common Trigrams
[('went looking for', 39), ('a symbol of', 23), ('he went looking', 20), ('she went looking', 19), ('looking for a', 18), ('was hungry so', 18), ('so she went', 17), ('so he went', 17), ('The result is', 17), ('result is a', 15)]



Let's create matrix with all the context sequences (ngrams) and next_word counts in our data. This gives us the numerator of the formula: $$ \text{Count(A B)}$$

In [ ]:
def get_ngram_count_matrix(stories, n):
    ngram_counts = defaultdict(Counter)

    for story in stories:
        for ngram in generate_ngrams(story, n+1):
            words = split_text(ngram)
            context = " ".join(words[:-1])
            next_word = words[-1]
            # if context not in ngram_counts:
            #     ngram_counts[context] = defaultdict(int)
            ngram_counts[context][next_word] += 1


    matrix_data = {context: dict(ngram_counts[context]) for context in ngram_counts}

    # Convert to DataFrame and fill missing values with 0
    df = pd.DataFrame(matrix_data).fillna(0).astype(int)


    return ngram_counts,df
#let's look at counts of bigrams
ngram_counts,df=get_ngram_count_matrix(stories,2)
df

,"Didier Drogba,","Drogba, a",a name,name that,that resonates,resonates with,with football,football fans,"fans worldwide,","worldwide, is",...,found a,"chai, rich","and cardamom,","cardamom, that",warmed him,him up,up and,and quenched,quenched his,his thirst
a,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
name,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
that,0,0,2,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
resonates,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
means,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Chinedu,",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oppressive,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
friend,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
handed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




**Step 3: Calculate $${\text{Count(A)}} $$**
Based on our formula, dividing these ngram counts by the total count of words in context (Step 3) will give us the conditional probabilities we're after.


Remember the conditional probability formula:

$$ P(\text{B} \mid \text{A}) = \frac{\text{Count(A B)}}{\text{Count(A)}} $$


Let's do that now!

In [ ]:
def build_ngram_model(ngram_counts, n):
    # Convert counts to probabilities
    ngram_model = {}
    for context, next_words in ngram_counts.items():
        total_count = sum(next_words.values())
        ngram_model[context] = {word: count / total_count for word, count in next_words.items()}

    return ngram_model

ngram_model = build_ngram_model(ngram_counts,2)

# Let's look at a few probabilities in our ngram model
sample_keys = list(ngram_model.keys())[:5]
for key in sample_keys:
    print(f"{key}: {ngram_model[key]}")

Didier Drogba,: {'a': 1.0}
Drogba, a: {'name': 1.0}
a name: {'that': 1.0}
name that: {'resonates': 0.5, 'means': 0.5}
that resonates: {'with': 1.0}



Let's understand the datatype of our ngram model. Our ngram model is a dictionary where:

- Keys: Represent the context (for example, a unigram "for", or bigram "looking for").

- Values: Represent another dictionary, keys of which are all candidates words observed in our dataset and values are their probabilities.

Below is an example of a dictionary within a dictionary.


In [ ]:
# Example: A dictionary of student grades
student_grades = {
    'Annie': {
        'Math': 95,
        'Science': 88,
        'History': 92
    },
    'Teju': {
        'Math': 85,
        'Science': 90,
        'History': 80
    }
}

# Accessing the dictionary for Teju
teju_grades = student_grades['Teju']
print("Teju's grades:", teju_grades)

# Extracting the subjects (keys) for Teju
subjects = list(teju_grades.keys())
print("Subjects for Teju:", subjects)

# Extracting the grades (values) for Teju
grades = list(teju_grades.values())
print("Grades for Teju:", grades)


Teju's grades: {'Math': 85, 'Science': 90, 'History': 80}
Subjects for Teju: ['Math', 'Science', 'History']
Grades for Teju: [85, 90, 80]


In [ ]:
#Using our model, for the context 'looking for', the result is another dictionary
#with candidate words as keys and their probabilities as values.
ngram_model['looking for']

{'the': 0.15384615384615385,
 'anything': 0.02564102564102564,
 'alternative': 0.02564102564102564,
 'scraps': 0.02564102564102564,
 'any': 0.02564102564102564,
 'a': 0.46153846153846156,
 'Banku': 0.02564102564102564,
 'Tella,': 0.02564102564102564,
 'Maafe,': 0.02564102564102564,
 'Umqombothi,': 0.02564102564102564,
 'sugarcane': 0.02564102564102564,
 'crispy': 0.02564102564102564,
 'warm': 0.02564102564102564,
 'Doro': 0.02564102564102564,
 'sambusa,': 0.02564102564102564,
 'dodo,': 0.02564102564102564,
 'Fura': 0.02564102564102564}

### Coding exercise:

Now, we need to write the code to predict the next word based on a given context.  We used `random.choices` in the previous course to randomly select an candidate word from a list based on specified weights (probabilities). We'll use it again here. In the previous course, the probabilities came from your mental model of english, this time around the probabilities will be based on the n-grams we have observed in our dataset. As a refresher, below is a sample usage of `random.choices`.


In [ ]:
import random

# Define a list of items
sample_candidate_words = ['apple', 'banana', 'cherry']

# Define corresponding weights for each fruit
# The probability of choosing each fruit is proportional to its weight.
weights = [0.2, 0.5, 0.3]

# Sample one fruit based on the weights.
# The 'k=1' parameter indicates that we're selecting one item.
chosen_fruit = random.choices(sample_candidate_words, weights=weights, k=1)[0]

print("Chosen fruit:", chosen_fruit)


Chosen fruit: apple



To use random choice for the context "looking for". You need to write code that performs the following steps:

Step 1: Extract candidate words for a given context from the ngram_model using the .keys() method.

Step 2: Extract the corresponding probability weights using the .values() method.

Step 1

In [ ]:
context = "looking for"
candidate_words=#enter code here
candidate_words

dict_keys(['the', 'anything', 'alternative', 'scraps', 'any', 'a', 'Banku', 'Tella,', 'Maafe,', 'Umqombothi,', 'sugarcane', 'crispy', 'warm', 'Doro', 'sambusa,', 'dodo,', 'Fura'])

In [ ]:
# @title Run me to test your code or get a hint. (Try not to peek this code until you've given it a good try!)
def test_candidate_words():
  hint = """
        Hint:
        ======
        The ngram model is a dictionary where each context maps to another dictionary of candidate words.
        To get a list of candidate words for a specific context, put the context in the model as ngram_model[context] and then use the .keys() method.
   """
  context = "looking for"

  try:
    if candidate_words==ngram_model[context].keys():
      print("Nice! Your answer looks correct.")
    else:
      print("\033[1m\033[91mSorry, your answer is not correct.\033[0m")
      give_hints = input("Would you like a hint? type Yes or No ")
      if give_hints.lower() in ['yes', "y"]:
        print(f"{hint}")
  except NameError:
          print("Hint: The variable 'candidate_words' is not defined.")
          print("You need to write some code after candidate_words=")

test_candidate_words()

Hint: The variable 'candidate_words' is not defined.
You need to write some code after candidate_words=


Step 2

In [ ]:
context = "looking for"
weights = #enter code here
weights

dict_values([0.15384615384615385, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.46153846153846156, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564, 0.02564102564102564])

In [ ]:
# @title Run me to test your code or get a hint. (Try not to peek this code until you've given it a good try!')
def test_weights():
  hint = """
        Hint:
        ======
        This is the same as the step above except this time around we are fetching the values, not keys. Try using .values() instead of .keys().
    """

  try:
    if list(weights) == list(ngram_model[context].values()):
      print("Nice! Your answer looks correct.")
    else:
      print("\033[1m\033[91mSorry, your answer is not correct.\033[0m")
      give_hints = input("Would you like a hint? type Yes or No ")
      if give_hints.lower() in ['yes', "y"]:
        print(f"{hint}")
  except NameError:
          print("Hint: The variable 'weights' is not defined.")
          print("You need to write some code after weights=")


test_weights()

Nice! Your answer looks correct.


Let's put these in random.choices and generate a next possible word.


In [ ]:
context = "looking for"
random.choices(list(candidate_words),weights=weights)[0]

'the'

Coding activity finished.




**Step 4: Generate text**

Now that we have a probability distribution over ngrams, let's use that to generate the next words given a prompt.
A prompt can be any phrase or word you want to give the model as a context or a starting point. A model can generate language by randomly selecting the next word based on their probabilities conditioned on preceding words (context) in the prompt. This is a simple but effective method for creating text that resembles the source material used to train the model.





In [ ]:
def generate_next_n_words(ngram_model, prompt, num_next_words, ngram):

    generated_words = split_text(prompt)  #split prompt into individual words

    for _ in range(num_next_words):
        context = generated_words[-(ngram):]  # Get last (n) words
        context = " ".join(context)
        if context in ngram_model:
            # Sample next word based on probabilities
            next_word = random.choices(
                list(ngram_model[context].keys()),
                weights=ngram_model[context].values()
            )[0]

            generated_words.append(next_word)
        else:
            print("No valid continuation found. Change the prompt or try a smaller ngram")  # Show when no next word is found
            break  # Stop if no valid continuation exists

    return " ".join(generated_words)

1. Run the following cell multiple times to see different results.
2. Try a simpler prompt, preferably one that uses phrases from the training data.
3. Try different values of n





In [ ]:
prompt="Jide was hungry so she went looking for"

n =2
num_next_words = 10 #generate next n words
generate_next_n_words(ngram_model,prompt,num_next_words,n)

'Jide was hungry so she went looking for Maafe, a rich palm nut soup from southern Nigeria. It'

The different results when running the cell multiple times are because the n-gram model picks the next word from a probability distribution randomly (also called *sampling*). More probable next words have a high probability of getting picked but are not guarenteed to be picked. This gives the output its stochasticity. Stochasticity means randomness or chance.

**What happens if you increase the context window i.e. the n in n-grams?**

Increasing the context window significantly impacts the ability to build a robust language model.  While it intuitively seems that a larger context (higher n) would lead to better accuracy by capturing more long-range dependencies in language,  it quickly runs into the problem of data sparsity (i.e. high dimensional array with many zero values).

With higher-order n-grams, the number of possible n-gram combinations explodes.  For example, moving from bigrams (n=2) to trigrams (n=3) drastically increases the possible combinations.  Even with massive datasets, many of these higher-order n-grams will never appear in the training corpus.  This results in many zero counts for the probabilities, making it impossible to estimate probabilities directly because P(w|context) = 0 if the specific n-gram (w, context) wasn't seen in the training data. Even though, advanced techniques like smoothing exist, the accuracy gains from a larger context window may not always outweigh the computational cost and potential for error introduced by sparse data.